# OneWeb

# Single Time

This cell will plot the satellites at a given time. There is currently a project to add the beam patterns for the closest satellites, but currently that saturates the image. Please keep addBeamMap False for now.

In [ ]:
import ephem
import datetime
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import pathlib
import re
from mplcursors import cursor  # separate package must be installed
import plotly.express as px
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
from math import sqrt,atan,pi,sin,cos
import pyproj
import geopy.distance
import datetime as dt
from datetime import datetime
from datetime import datetime, timedelta

############
## INPUTS ##
############

gatech = ephem.Observer()
gatech.lon = '-107.618330'
gatech.lat = '34.078815'
gatech.elevation = 0 # meters???
# gatech.date = '2023/11/27 19:11:05'
gatech.date = '2024/2/19 21:37:19'
addBeamMap = True
checkWithinDistance = 750 # km
GBO_included = True
    
############
## INPUTS ##
############

def latCov(latStr):
    deg, minutes, seconds =  re.split('[:]', latStr)
    return (float(deg) + float(minutes)/60 + float(seconds)/(60*60))
def lonCov(lonStr):
    deg, minutes, seconds =  re.split('[:]', lonStr)
    return (-1*float(deg) + float(minutes)/60 + float(seconds)/(60*60))*-1

def squareCoords(center_lat, center_lon, width, height, iD="both"):
    # ID either both, above, or below
    #wideth/height in meters
    geod = pyproj.Geod(ellps='WGS84')
    inclination = 90-87.9 # degrees
    inclination_rad = inclination*pi/180

    rect_diag = sqrt( width**2 + height**2 )

    azimuth1 = atan(width/height) + inclination_rad
    azimuth2 = atan(-width/height) + inclination_rad
    azimuth3 = atan(width/height)+pi+ inclination_rad  # first point + 180 degrees
    azimuth4 = atan(-width/height)+pi+ inclination_rad # second point + 180 degrees


    pt1_lon, pt1_lat, _ = geod.fwd(center_lon, center_lat, azimuth1*180/pi, rect_diag)
    pt2_lon, pt2_lat, _ = geod.fwd(center_lon, center_lat, azimuth2*180/pi, rect_diag)
    pt3_lon, pt3_lat, _ = geod.fwd(center_lon, center_lat, azimuth3*180/pi, rect_diag)
    pt4_lon, pt4_lat, _ = geod.fwd(center_lon, center_lat, azimuth4*180/pi, rect_diag)
    
    if iD == "above":
        if pt1_lat < center_lat:
            pt1_lat = center_lat
        if pt2_lat < center_lat:
            pt2_lat = center_lat
        if pt3_lat < center_lat:
            pt3_lat = center_lat
        if pt4_lat < center_lat:
            pt4_lat = center_lat
    if iD == "below":
        if pt1_lat > center_lat:
            pt1_lat = center_lat
        if pt2_lat > center_lat:
            pt2_lat = center_lat
        if pt3_lat > center_lat:
            pt3_lat = center_lat
        if pt4_lat > center_lat:
            pt4_lat = center_lat
    
        

    wkt_point = 'POINT (%.6f %.6f)' % (center_lon, center_lat)
    wkt_poly = 'POLYGON (( %.6f %.6f, %.6f %.6f, %.6f %.6f, %.6f %.6f, %.6f %.6f ))' % (pt1_lon, pt1_lat, pt2_lon, pt2_lat, pt3_lon, pt3_lat, pt4_lon, pt4_lat, pt1_lon, pt1_lat)    
    
    return [pt1_lat, pt2_lat, pt3_lat, pt4_lat], [pt1_lon, pt2_lon, pt3_lon, pt4_lon]

distance_ontrack = [0, 10.47, 20.94, 25.13, 33.5, 40.82, 48.15, 60.69, 69.04, 1515] #km
distance_crosstrack = [0, 125.21, 287.58, 384.24, 542.09, 671.77, 788.5, 892.27, 1091.97, 1515] #km
gradient = [0, -.2, -1, -4, -6, -8, -10, -15, -20, -20] # dB
df_pattern = pd.DataFrame(
    {'Distance on Track': distance_ontrack,
     'Distanace cross Track': distance_crosstrack,
     'Loss (dB)': gradient
    })

with open('TLE_files/OnewebSats6.27.23.txt') as f:
    lines = f.readlines()
    

#####
#####
#####

timeList = []
names = []
tle1 = []
tle2 = []

latListMaster = []
longListMaster = []

for i in lines:

    if i[0] == "O":
        names.append(i)
    if i[0] == "1":
        tle1.append(i)
    if i[0] == "2":
        tle2.append(i)


for i in range(len(names)):
    name = names[i]
    line1 = tle1[i]
    line2 = tle2[i]
    
    tle_rec = ephem.readtle(name, line1, line2)
    tle_rec.compute(gatech)
    latStr = str(tle_rec.sublat)
    lonStr = str(tle_rec.sublong)
    
    latListMaster.append(latCov(latStr))
    longListMaster.append(lonCov(lonStr))
    timeList.append(str(gatech.date))
    
##########
# Plotting
##########


myData = pd.read_excel(r"2023.12.13-VLA Antenna Locations.xlsx")
myDataVLA = myData[myData["Telescope"] == "VLA"]

df = pd.DataFrame(
    {'Latitude': latListMaster,
     'Longitude': longListMaster,
     'Satellite Name': names,
     'Time (UTC)': timeList
    })
df['Object'] = 'OneWeb'

fig = px.scatter_mapbox(df, 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Satellite Name", 
                        hover_data=["Satellite Name", 'Time (UTC)'],
                        center=dict(lon=-90.618330, lat=37.078815),
                        zoom=2.9, # 4
                        height=800,
                        color = 'Object',
                        width=800) 

fig.update_traces(marker={'size': 10})

fig.add_scattermapbox(lat = myDataVLA["Latitude"] ,lon = myDataVLA["Longitude"] ,marker_size = 6, 
                      marker_color = 'rgb(0, 0, 0)',name = 'VLA (A config)', showlegend = True)

if GBO_included == True:
    fig.add_scattermapbox(lat = [38.43299223947407], lon = [-79.83961160955101] ,marker_size = 12,
                      marker_color = 'rgb(150, 168, 50)',name = 'GBT', showlegend = True)

c = 0
if addBeamMap == True:
    for j in range(len(latListMaster)):
        coords_1 = (34.078815, -107.618330)
        coords_2 = (latListMaster[j], longListMaster[j])
        distBetweenPoints = geopy.distance.geodesic(coords_1, coords_2).km
        
        if GBO_included == True:
            coords_12 = (38.43299223947407, -79.83961160955101)
            coords_22 = (latListMaster[j], longListMaster[j])
            distBetweenPoints2 = geopy.distance.geodesic(coords_12, coords_22).km
        
        if (distBetweenPoints < checkWithinDistance or distBetweenPoints2 < checkWithinDistance):
            # These are oriented the other way
#             square_meanBeam_4dB = squareCoords(latListMaster[j], longListMaster[j], width = 934000, height = df_pattern['Distanace cross Track'][3]*1e3, iD="both") # 3 is up to 4 db fall off
#             square_meanBeam = squareCoords(latListMaster[j], longListMaster[j], width = 934000, height = df_pattern['Distanace cross Track'][1]*1e3, iD="both") # True main beam
#             square_meanBeam_all = squareCoords(latListMaster[j], longListMaster[j], width = 934000, height = 934000, iD="both") # True main beam
            square_meanBeam_4dB = squareCoords(latListMaster[j], longListMaster[j], width = df_pattern['Distanace cross Track'][3]*1e3, height = 934000, iD="both") # 3 is up to 4 db fall off
            square_meanBeam = squareCoords(latListMaster[j], longListMaster[j], width = df_pattern['Distanace cross Track'][1]*1e3, height = 934000, iD="both") # True main beam
            square_meanBeam_all = squareCoords(latListMaster[j], longListMaster[j], width = 934000, height = 934000, iD="both") # True main beam

            if c < 1:
                c += 1
                fig.add_scattermapbox(lat = square_meanBeam_all[0],lon = square_meanBeam_all[1] ,marker_size = 6, fill="toself", 
                            marker_color = 'rgba(200, 0, 0, 0.001)', fillcolor="rgba(250, 0, 0, .1)",name='All Beam', showlegend=True)
                fig.add_scattermapbox(lat = square_meanBeam_4dB[0],lon = square_meanBeam_4dB[1] ,marker_size = 6, fill="toself", 
                            marker_color = 'rgba(250, 0, 0, 0.001)', fillcolor="rgba(250, 0, 0, .25)", name='Main Beam - 4 dB', showlegend=True)
                fig.add_scattermapbox(lat = square_meanBeam[0],lon = square_meanBeam[1] ,marker_size = 6, fill="toself", 
                            marker_color = 'rgba(200, 0, 0, 0.001)', fillcolor="rgba(250, 0, 0, .5)",name='Main Beam', showlegend=True) 
            else:
                fig.add_scattermapbox(lat = square_meanBeam_all[0],lon = square_meanBeam_all[1] ,marker_size = 6, fill="toself", 
                            marker_color = 'rgba(200, 0, 0, 0.001)', fillcolor="rgba(250, 0, 0, .1)", showlegend=False)
                fig.add_scattermapbox(lat = square_meanBeam_4dB[0],lon = square_meanBeam_4dB[1] ,marker_size = 6, fill="toself", 
                            marker_color = 'rgba(250, 0, 0, 0.001)', fillcolor="rgba(250, 0, 0, .25)", showlegend=False)
                fig.add_scattermapbox(lat = square_meanBeam[0],lon = square_meanBeam[1] ,marker_size = 6, fill="toself", 
                            marker_color = 'rgba(200, 0, 0, 0.001)', fillcolor="rgba(250, 0, 0, .5)", showlegend=False)
                

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(
    autosize=False,
    width=1200,
    height=800
)
fig.update_layout(
    title_x=0.5,
    title_y=0.95,
    title_text=f"OneWeb Satellite Positions (and beams within {checkWithinDistance} km) at {gatech.date} UTC",
    margin={"l": 0, "r": 0, "b": 0, "t": 80}
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))


fig.data = fig.data[::-1]
fig.show()
myDate = str(gatech.date)
myDate1 = myDate.replace("/", "_")
myDate2 = myDate1.replace(":", ".")
if addBeamMap == True:
    newFolder = myDate2.split(" ")[0]
    save_file_html = f"Outputs\GlobalPositions\OneWeb_{newFolder}\OneWebPositions VLA {myDate2} with Beam Map.html"
    save_file_pdf = f"Outputs\GlobalPositions\OneWeb_{newFolder}\OneWebPositions VLA {myDate2} with Beam Map.pdf"
    Path(f"Outputs\GlobalPositions\OneWeb_{newFolder}").mkdir(parents=True, exist_ok=True)
    fig.write_html(save_file_html)
    fig.write_image(save_file_pdf)
else:
    newFolder = myDate2.split(" ")[0]
    save_file_html = f"Outputs\GlobalPositions\OneWeb_{newFolder}\OneWebPositions VLA {myDate2}.html"
    save_file_pdf = f"Outputs\GlobalPositions\OneWeb_{newFolder}\OneWebPositions VLA {myDate2}.pdf"
    Path(f"Outputs\GlobalPositions\OneWeb_{newFolder}").mkdir(parents=True, exist_ok=True)
    fig.write_html(save_file_html)
    fig.write_image(save_file_pdf)    

# Range of Times

This is the same as above, but will plot a time range. This currently doesnt have beam maps added, as it will overload the plot.

In [ ]:
############
## INPUTS ##
############

startUTC = '2024/2/19 21:09:00'
endUTC = '2024/2/19 21:11:00'
            

gatech = ephem.Observer()
gatech.lon = '-107.618330'
gatech.lat = '34.078815'
gatech.elevation = 0 # meters???
GBO_included = True

############
## INPUTS ##
############

date_format = '%Y/%m/%d %H:%M:%S'

UTC_range = []

date = startUTC.split(" ")[0]
startHour = startUTC.split(" ")[1].split(':')[0]
startmin = startUTC.split(" ")[1].split(':')[1]
startsec = startUTC.split(" ")[1].split(':')[2]
endHour = endUTC.split(" ")[1].split(':')[0]
endmin = endUTC.split(" ")[1].split(':')[1]
endsec = endUTC.split(" ")[1].split(':')[2]

a = dt.datetime(int(date.split("/")[0]),int(date.split("/")[1]),int(date.split("/")[2]),int(startHour),int(startmin),int(startsec))
b = dt.datetime(int(date.split("/")[0]),int(date.split("/")[1]),int(date.split("/")[2]),int(endHour),int(endmin),int(endsec))

totSec = int((b-a).total_seconds())

for i in range(totSec+1):
    date_obj = datetime.strptime(startUTC, date_format)
    date_objs = date_obj + timedelta(0,i)
    UTC_range.append(date_objs)
    

def latCov(latStr):
    deg, minutes, seconds =  re.split('[:]', latStr)
    return (float(deg) + float(minutes)/60 + float(seconds)/(60*60))
def lonCov(lonStr):
    deg, minutes, seconds =  re.split('[:]', lonStr)
    return (-1*float(deg) + float(minutes)/60 + float(seconds)/(60*60))*-1

with open('TLE_files/OnewebSats6.27.23.txt') as f:
    lines = f.readlines()
    

#####
#####
#####

timeList = []
names = []
tle1 = []
tle2 = []
latListMaster = []
longListMaster = []

for i in lines:
    if i[0] == "O":
        names.append(i)
    if i[0] == "1":
        tle1.append(i)
    if i[0] == "2":
        tle2.append(i)
            
for tim in UTC_range:
    gatech.date = tim

    for i in range(len(names)):
        name = names[i]
        line1 = tle1[i]
        line2 = tle2[i]

        tle_rec = ephem.readtle(name, line1, line2)
        tle_rec.compute(gatech)
        latStr = str(tle_rec.sublat)
        lonStr = str(tle_rec.sublong)

        latListMaster.append(latCov(latStr))
        longListMaster.append(lonCov(lonStr))
        timeList.append(str(tim))

##########
# Plotting
##########


myData = pd.read_excel(r"2023.12.13-VLA Antenna Locations.xlsx")
myDataVLA = myData[myData["Telescope"] == "VLA"]

names = names*(totSec+1)
df = pd.DataFrame(
    {'Latitude': latListMaster,
     'Longitude': longListMaster,
     'Satellite Name': names,
     'Time (UTC)': timeList
    })
df['Object'] = 'OneWeb'

fig = px.scatter_mapbox(df, 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Satellite Name", 
                        hover_data=["Satellite Name", "Time (UTC)"],
                        center=dict(lon=-90.618330, lat=37.078815),
                        zoom=3.5,
                        height=800,
                        color = 'Object',
                        width=800)

fig.update_traces(marker={'size': 10})

fig.add_scattermapbox(lat = myDataVLA["Latitude"] ,lon = myDataVLA["Longitude"] ,marker_size = 6, 
                      marker_color = 'rgb(0, 0, 0)',name = 'VLA (A config)', showlegend = True)

if GBO_included == True:
    fig.add_scattermapbox(lat = [38.43299223947407], lon = [-79.83961160955101] ,marker_size = 12,
                      marker_color = 'rgb(150, 168, 50)',name = 'GBT', showlegend = True)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(
    autosize=False,
    width=1200,
    height=800,
)
fig.update_layout(
    title_x=0.5,
    title_y=0.95,
    title_text=f"OneWeb Satellite Positions at {startUTC} to {endUTC} UTC",
    margin={"l": 0, "r": 0, "b": 0, "t": 80}
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()
myDate = startUTC + " to " + endUTC
myDate1 = myDate.replace("/", "_")
myDate2 = myDate1.replace(":", ".")
newFolder = myDate2.split(" ")[0]
save_file_html = f"Outputs\SatelliteAnalysis\GlobalPositions\OneWeb_{newFolder}\OneWebPositions VLA {myDate2}.html"
save_file_pdf = f"Outputs\SatelliteAnalysis\GlobalPositions\OneWeb_{newFolder}\OneWebPositions VLA {myDate2}.pdf"
Path(f"Outputs\SatelliteAnalysis\GlobalPositions\OneWeb_{newFolder}").mkdir(parents=True, exist_ok=True)
fig.write_html(save_file_html)
fig.write_image(save_file_pdf)

# Starlink Single Sat

In [ ]:
############
## INPUTS ##
############

startUTC = '2024/3/28 16:58:00'
endUTC = '2024/3/28 16:59:45'

StarlinkName = 'STARLINK-11072'
            

gatech = ephem.Observer()
gatech.lon = '-107.618330'
gatech.lat = '34.078815'
gatech.elevation = 0 # meters???
GBO_included = True

############
## INPUTS ##
############

date_format = '%Y/%m/%d %H:%M:%S'

UTC_range = []

date = startUTC.split(" ")[0]
startHour = startUTC.split(" ")[1].split(':')[0]
startmin = startUTC.split(" ")[1].split(':')[1]
startsec = startUTC.split(" ")[1].split(':')[2]
endHour = endUTC.split(" ")[1].split(':')[0]
endmin = endUTC.split(" ")[1].split(':')[1]
endsec = endUTC.split(" ")[1].split(':')[2]

a = dt.datetime(int(date.split("/")[0]),int(date.split("/")[1]),int(date.split("/")[2]),int(startHour),int(startmin),int(startsec))
b = dt.datetime(int(date.split("/")[0]),int(date.split("/")[1]),int(date.split("/")[2]),int(endHour),int(endmin),int(endsec))

totSec = int((b-a).total_seconds())

for i in range(totSec+1):
    date_obj = datetime.strptime(startUTC, date_format)
    date_objs = date_obj + timedelta(0,i)
    UTC_range.append(date_objs)
    

def latCov(latStr):
    deg, minutes, seconds =  re.split('[:]', latStr)
    return (float(deg) + float(minutes)/60 + float(seconds)/(60*60))
def lonCov(lonStr):
    deg, minutes, seconds =  re.split('[:]', lonStr)
    return (-1*float(deg) + float(minutes)/60 + float(seconds)/(60*60))*-1

with open('TLE_files/StarlinkSats6.27.23.txt') as f:
    lines = f.readlines()
    

#####
#####
#####

timeList = []
names = []
tle1 = []
tle2 = []
latListMaster = []
longListMaster = []

for i in lines:
    if i[0] == "S":
        if StarlinkName in i:
            names.append(i)
    if i[0] == "1":
        tle1.append(i)
    if i[0] == "2":
        tle2.append(i)
            
for tim in UTC_range:
    gatech.date = tim

    for i in range(len(names)):
        name = names[i]
        line1 = tle1[i]
        line2 = tle2[i]

        tle_rec = ephem.readtle(name, line1, line2)
        tle_rec.compute(gatech)
        latStr = str(tle_rec.sublat)
        lonStr = str(tle_rec.sublong)

        latListMaster.append(latCov(latStr))
        longListMaster.append(lonCov(lonStr))
        timeList.append(str(tim))

##########
# Plotting
##########


myData = pd.read_excel(r"2023.12.13-VLA Antenna Locations.xlsx")
myDataVLA = myData[myData["Telescope"] == "VLA"]

names = names*(totSec+1)
df = pd.DataFrame(
    {'Latitude': latListMaster,
     'Longitude': longListMaster,
     'Satellite Name': names,
     'Time (UTC)': timeList
    })
df['Object'] = 'Starlink'

fig = px.scatter_mapbox(df, 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Satellite Name", 
                        hover_data=["Satellite Name", "Time (UTC)"],
                        center=dict(lon=-90.618330, lat=37.078815),
                        zoom=3.5,
                        height=800,
                        color = 'Object',
                        width=800)

fig.update_traces(marker={'size': 10})

fig.add_scattermapbox(lat = myDataVLA["Latitude"] ,lon = myDataVLA["Longitude"] ,marker_size = 6, 
                      marker_color = 'rgb(0, 0, 0)',name = 'VLA (A config)', showlegend = True)

if GBO_included == True:
    fig.add_scattermapbox(lat = [38.43299223947407], lon = [-79.83961160955101] ,marker_size = 12,
                      marker_color = 'rgb(150, 168, 50)',name = 'GBT', showlegend = True)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(
    autosize=False,
    width=1200,
    height=800,
)
fig.update_layout(
    title_x=0.5,
    title_y=0.95,
    title_text=f"Starlink Satellite Position {StarlinkName} at {startUTC} to {endUTC} UTC",
    margin={"l": 0, "r": 0, "b": 0, "t": 80}
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()
myDate = startUTC + " to " + endUTC
myDate1 = myDate.replace("/", "_")
myDate2 = myDate1.replace(":", ".")
newFolder = myDate2.split(" ")[0]
# save_file_html = f"Outputs\SatelliteAnalysis\GlobalPositions\OneWeb_{newFolder}\OneWebPositions VLA {myDate2}.html"
# save_file_pdf = f"Outputs\SatelliteAnalysis\GlobalPositions\OneWeb_{newFolder}\OneWebPositions VLA {myDate2}.pdf"
# Path(f"Outputs\SatelliteAnalysis\GlobalPositions\OneWeb_{newFolder}").mkdir(parents=True, exist_ok=True)
# fig.write_html(save_file_html)
# fig.write_image(save_file_pdf)

# Satellite inputs for all TLEs I have

In [ ]:
import ephem
import datetime
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import pathlib
import re
from mplcursors import cursor  # separate package must be installed
import plotly.express as px
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
from math import sqrt,atan,pi,sin,cos
import pyproj
import geopy.distance
import datetime as dt
from datetime import datetime
from datetime import datetime, timedelta
from random import randint
import numpy as np


############
## INPUTS ##
############

startUTC = '2024/04/16 00:20:28'
endUTC = '2024/04/16 00:37:27'
            

gatech = ephem.Observer()
gatech.lon = '-79.8398333330'
gatech.lat = '38.4331111110'
gatech.elevation = 0 # meters???
GBO_included = True
plotTimes = False

# filename = "TerraMission"
# filename = "SAR_public"
# filename = "X-Band_TLE_satDump"
filename = "ManuallyConcatenated"  # this may have duplicates??


############
## INPUTS ##
############

date_format = '%Y/%m/%d %H:%M:%S'

UTC_range = []

date = startUTC.split(" ")[0]
startHour = startUTC.split(" ")[1].split(':')[0]
startmin = startUTC.split(" ")[1].split(':')[1]
startsec = startUTC.split(" ")[1].split(':')[2]
endHour = endUTC.split(" ")[1].split(':')[0]
endmin = endUTC.split(" ")[1].split(':')[1]
endsec = endUTC.split(" ")[1].split(':')[2]

a = dt.datetime(int(date.split("/")[0]),int(date.split("/")[1]),int(date.split("/")[2]),int(startHour),int(startmin),int(startsec))
b = dt.datetime(int(date.split("/")[0]),int(date.split("/")[1]),int(date.split("/")[2]),int(endHour),int(endmin),int(endsec))

totSec = int((b-a).total_seconds())

for i in range(totSec+1):
    date_obj = datetime.strptime(startUTC, date_format)
    date_objs = date_obj + timedelta(0,i)
    UTC_range.append(date_objs)
    

def latCov(latStr):
    deg, minutes, seconds =  re.split('[:]', latStr)
    return (float(deg) + float(minutes)/60 + float(seconds)/(60*60))
def lonCov(lonStr):
    deg, minutes, seconds =  re.split('[:]', lonStr)
    return (-1*float(deg) + float(minutes)/60 + float(seconds)/(60*60))*-1
def LatLongToAzEl(latObs, longObs, latObj, longObj, heightObj):
    pass


filenameUse = "TLE_files/" + filename + ".txt"
with open(filenameUse) as f:
    lines = f.readlines()
    

#####
#####
#####

timeList = []
names = []
tle1 = []
tle2 = []
latListMaster = []
longListMaster = []
altList = []
azList = []

for i in lines:
    if i[0].isalpha():
        names.append(i)
    if i[0] == "1":
        tle1.append(i)
    if i[0] == "2":
        tle2.append(i)
            
for tim in UTC_range:
    gatech.date = tim

    for i in range(len(names)):
        name = names[i]
        line1 = tle1[i]
        line2 = tle2[i]

        tle_rec = ephem.readtle(name, line1, line2)
        tle_rec.compute(gatech)
        latStr = str(tle_rec.sublat)
        lonStr = str(tle_rec.sublong)
        
        altStr = str(tle_rec.alt)
        azStr = str(tle_rec.az)

        latListMaster.append(latCov(latStr))
        longListMaster.append(lonCov(lonStr))
        timeList.append(str(tim))
        if '-' in altStr:
            altList.append(lonCov(altStr))
        else:
            altList.append(latCov(altStr))    
        if '-' in azStr:
            azList.append(lonCov(azStr))
        else:
            azList.append(latCov(azStr))

##########
# Plotting
##########


myData = pd.read_excel(r"2023.12.13-VLA Antenna Locations.xlsx")
myDataVLA = myData[myData["Telescope"] == "VLA"]

names = names*(totSec+1)

UniqueSatName = list(np.unique(names))
color = []
colorsList = []
n = len(UniqueSatName)

for i in range(n):
    color.append('#%06X' % randint(0, 0xFFFFFF))
    
for satName in names:
    myIndex = UniqueSatName.index(satName)
    colorsList.append(color[myIndex])
    
df = pd.DataFrame(
    {'Latitude': latListMaster,
     'Longitude': longListMaster,
     'Satellite Name': names,
     'Time (UTC)': timeList,
     'colors': colorsList
    })
df['Object'] = 'Satellite'

dfMasekd_colors = df[['Satellite Name','colors']].drop_duplicates()
for item, row in dfMasekd_colors.iterrows():
    dfMasekd_colors['Satellite Name'][item] = dfMasekd_colors['Satellite Name'][item].replace("\n", "")

fig = px.scatter_mapbox(df, 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Satellite Name", 
                        hover_data=["Satellite Name", "Time (UTC)"],
                        center=dict(lon=-90.618330, lat=37.078815),
                        zoom=3.5,
                        height=800,
                        color = 'Object',
                        width=800)

fig.update_traces(marker={'size': 10})

fig.add_scattermapbox(lat = myDataVLA["Latitude"] ,lon = myDataVLA["Longitude"] ,marker_size = 6, 
                      marker_color = 'rgb(0, 0, 0)',name = 'VLA (A config)', showlegend = True)

if GBO_included == True:
    fig.add_scattermapbox(lat = [38.43299223947407], lon = [-79.83961160955101] ,marker_size = 12,
                      marker_color = 'rgb(150, 168, 50)',name = 'GBT', showlegend = True)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(
    autosize=False,
    width=1200,
    height=800,
)
fig.update_layout(
    title_x=0.5,
    title_y=0.95,
    title_text=f"{filename} Satellite Positions at {startUTC} to {endUTC} UTC",
    margin={"l": 0, "r": 0, "b": 0, "t": 80}
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()
myDate = startUTC + " to " + endUTC
myDate1 = myDate.replace("/", "_")
myDate2 = myDate1.replace(":", ".")
newFolder = myDate2.split(" ")[0]
save_file_html = f"Outputs\SatelliteAnalysis\GlobalPositions\_{filename}_{newFolder}\_{filename}_{myDate2}.html"
save_file_pdf = f"Outputs\SatelliteAnalysis\GlobalPositions\_{filename}_{newFolder}\_{filename}_{myDate2}.pdf"
Path(f"Outputs\SatelliteAnalysis\GlobalPositions\_{filename}_{newFolder}").mkdir(parents=True, exist_ok=True)
fig.write_html(save_file_html)
fig.write_image(save_file_pdf)

In [ ]:
from matplotlib.lines import Line2D

fig,ax = plt.subplots(1)
    
ax.scatter(azList, altList, s=5, c=df['colors'])
    
ax.set_xlim(0,360)
ax.set_ylim(-20, 90)
ax.set_title(f"{filename}/GBT on {myDate2}")
ax.set_xlabel("Azimuth")
ax.set_ylabel("Altitude")
ax.axhline(0, c='red', label='horizon')

newLabels = []
for j in UniqueSatName:
    newstr = j.replace("\n", "")
    newLabels.append(newstr)
    
lines = [Line2D([0], [0], color=c, linewidth=3, linestyle='-') for c in dfMasekd_colors['colors']]
if len(dfMasekd_colors['colors']) > 12:
    plt.legend(lines, dfMasekd_colors['Satellite Name'], ncol=2, loc='center left', fontsize = 7, bbox_to_anchor=(1, 0.6))  
else:
    plt.legend(lines, dfMasekd_colors['Satellite Name'], ncol=1, loc='center left', fontsize = 7, bbox_to_anchor=(1, 0.6))  
    
    
# This needs work...173 allows for no skips...but something cna be better
if plotTimes == True:
    n = timeList
    myCount = 173
    for i, txt in enumerate(n):
        if altList[i] > 0:
            if i % myCount == 0:
                txt = txt.split(" ")[1]
                ax.scatter(azList[i], altList[i], s=20, color='Black')
                ax.annotate(txt, (azList[i], altList[i]), xytext=(-25,25), textcoords='offset points', ha='center', va='bottom',fontsize=12,arrowprops=dict(arrowstyle='->', color='black', linewidth=2))


plt.savefig(f"Outputs\SatelliteAnalysis\AboveHorizon\_{filename}_{newFolder}\AzAlt_{filename}_{myDate2}.pdf", bbox_inches='tight')